In [ ]:
# ==========================================
# 1. 라이브러리 설치 
# ==========================================
!pip install pytorch-tabnet imbalanced-learn

import pandas as pd
import numpy as np
import torch
import copy
from pytorch_tabnet.tab_model import TabNetClassifier
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from sklearn.metrics import f1_score, classification_report, accuracy_score
from imblearn.over_sampling import SMOTE
from sklearn.utils.class_weight import compute_class_weight
from google.colab import drive

# ==========================================
# 2. 구글 드라이브 연결 및 데이터 로드
# ==========================================
drive.mount('/content/drive')

# 경로 설정
base_path = '/content/drive/MyDrive/AiWemeet/data/'
print("\n 데이터를 불러오는 중입니다...")

try:
    # 엑셀 파일 로드
    df_screen = pd.read_excel(base_path + 'screening_data_1001.xlsx')
    df_snsb = pd.read_excel(base_path + 'SNSB_1000.xlsx')
    df_apoe = pd.read_excel(base_path + 'APOE_982.xlsx')
    df_pet = pd.read_excel(base_path + 'PET_result_300.xlsx')

    # MRI 데이터 (3개 시트 로드)
    mri_path = base_path + 'mri_roi_992.xlsx'
    df_mri_sub = pd.read_excel(mri_path, sheet_name='subcortical')
    df_mri_surf = pd.read_excel(mri_path, sheet_name='surface')
    df_mri_thick = pd.read_excel(mri_path, sheet_name='thickness')
    print(" 모든 파일을 성공적으로 불러왔습니다!")

except FileNotFoundError:
    print(" [에러] 파일을 찾을 수 없습니다. 경로를 확인해주세요.")
    raise

# ==========================================
# 3. 데이터 통합 
# ==========================================
# ID 컬럼명 통일
if 'SubjectID' in df_screen.columns:
    df_screen = df_screen.rename(columns={'SubjectID': 'ID'})

dfs = [df_snsb, df_apoe, df_pet, df_mri_sub, df_mri_surf, df_mri_thick]
for df in dfs:
    if 'Subject ID' in df.columns:
        df.rename(columns={'Subject ID': 'ID'}, inplace=True)
    elif 'Subject_ID' in df.columns:
        df.rename(columns={'Subject_ID': 'ID'}, inplace=True)

# MRI 병합
drop_cols = ['순번', 'MRI ID', 'NO']
df_mri = df_mri_sub.drop(columns=drop_cols, errors='ignore') \
    .merge(df_mri_surf.drop(columns=drop_cols, errors='ignore'), on='ID', how='outer') \
    .merge(df_mri_thick.drop(columns=drop_cols, errors='ignore'), on='ID', how='outer')

# 전체 통합
df_master = df_screen.merge(df_snsb.drop(columns=['순번'], errors='ignore'), on='ID', how='left') \
                     .merge(df_apoe.drop(columns=['NO', '순번'], errors='ignore'), on='ID', how='left') \
                     .merge(df_mri, on='ID', how='left') \
                     .merge(df_pet.drop(columns=['순번'], errors='ignore'), on='ID', how='left')

# ==========================================
# 4. 데이터 전처리 (Preprocessing)
# ==========================================
# Target 매핑
target_mapping = {'CN': 0, 'SCD': 1, 'MCI': 2, 'Dem': 3}
df_master['target'] = df_master['DIA_01'].map(target_mapping)
df_master = df_master.dropna(subset=['target'])

# 불필요 컬럼 제거
cols_to_drop = ['ID', 'SubjectID', 'Num', 'Selection_num', 'test_day', 'Date',
                'DIA_01', 'target', 'MRI_ID', 'PET ID', '검사일자', '순번']
feature_cols = [c for c in df_master.columns if c not in cols_to_drop]

# 파생변수 생성 (PET 유무, APOE e4)
if '아밀로이드 베타' in df_master.columns:
    df_master['has_PET'] = df_master['아밀로이드 베타'].notnull().astype(int)
    feature_cols.append('has_PET')

def check_e4(x):
    if pd.isna(x): return 0
    return 1 if 'e4' in str(x).lower() else 0
if 'APOE' in df_master.columns:
    df_master['APOE_e4'] = df_master['APOE'].apply(check_e4)
    feature_cols.append('APOE_e4')

# 결측치 처리
df_train = df_master.copy()
for col in feature_cols:
    if df_train[col].dtype in ['int64', 'float64']:
        mean_val = df_train[col].mean()
        df_train[col] = df_train[col].fillna(0 if pd.isna(mean_val) else mean_val)
    else:
        df_train[col] = df_train[col].fillna("Unknown")
        le = LabelEncoder()
        df_train[col] = le.fit_transform(df_train[col].astype(str))

# 잔여 NaN 처리
if df_train[feature_cols].isnull().sum().sum() > 0:
    df_train[feature_cols] = df_train[feature_cols].fillna(0)

X = df_train[feature_cols].values.astype('float32')
y = df_train['target'].values.astype(int)

# ==========================================
# 5. 데이터 분할 및 SMOTE 적용
# ==========================================
X_train, X_temp, y_train, y_temp = train_test_split(X, y, test_size=0.3, stratify=y, random_state=42)
X_valid, X_test, y_valid, y_test = train_test_split(X_temp, y_temp, test_size=0.5, stratify=y_temp, random_state=42)

print("\n SMOTE로 데이터 불균형 해결 중...")
smote = SMOTE(k_neighbors=1, random_state=42)
X_train_res, y_train_res = smote.fit_resample(X_train, y_train)


# test_data 따로 저장

In [ ]:
import numpy as np
import os

# 1. 저장할 위치 (구글 드라이브 data 폴더)
save_path = '/content/drive/MyDrive/AiWemeet/data/' 

# 2. 저장 실행
print("테스트 데이터 저장")
np.save(save_path + 'X_test_data.npy', X_test)
np.save(save_path + 'y_test_data.npy', y_test)

print(f"저장 위치: {save_path}")

In [ ]:

# ==========================================
# 6. 앙상블 학습 시작 (5회 반복)
# ==========================================
print("\n 앙상블 학습을 시작합니다! (총 5회 반복)")
seeds = [42, 2023, 2024, 777, 999]
preds_test_probs = []

for i, seed in enumerate(seeds):
    print(f"\n[Model {i+1}/5] 학습 중 (Seed={seed})...")

    clf = TabNetClassifier(
        n_d=64, n_a=64, n_steps=5,
        gamma=1.2,
        n_independent=2, n_shared=2,
        lambda_sparse=1e-4,
        optimizer_fn=torch.optim.Adam,
        optimizer_params=dict(lr=1e-2),
        scheduler_params={"step_size":20, "gamma":0.9},
        scheduler_fn=torch.optim.lr_scheduler.StepLR,
        mask_type='entmax',
        seed=seed,
        verbose=0
    )

    clf.fit(
        X_train=X_train_res, y_train=y_train_res,
        eval_set=[(X_valid, y_valid)],
        eval_name=['valid'],
        eval_metric=['balanced_accuracy'],
        max_epochs=150,
        patience=40,
        batch_size=256,
        virtual_batch_size=32,
        num_workers=0,
        drop_last=False,
        weights=1
    )

    pred_prob = clf.predict_proba(X_test)
    preds_test_probs.append(pred_prob)
    print(f"  -> Best Valid Score: {clf.best_cost:.4f}")

# ==========================================
# 7. 최종 결과 종합 
# ==========================================
print("\n 결과 종합 중...")
avg_preds_proba = np.mean(preds_test_probs, axis=0)
final_preds = np.argmax(avg_preds_proba, axis=1)

final_f1 = f1_score(y_test, final_preds, average='macro')
print("\n" + "="*40)
print(f" 앙상블 최종 Macro F1 Score: {final_f1:.4f}")
print("="*40)
print(classification_report(y_test, final_preds, target_names=['CN', 'SCD', 'MCI', 'Dem'], zero_division=0))

In [ ]:
# ==========================================
# 8. [성능 극대화] 임계값 최적화
# ==========================================
import numpy as np
from sklearn.metrics import f1_score, classification_report

print("\n🔧 SCD(주관적 인지저하) 클래스 구출 작전을 시작합니다...")

# 5개 모델이 예측한 확률 평균값
# avg_preds_proba: [CN확률, SCD확률, MCI확률, Dem확률]


best_score = 0
best_th = 0
final_optimized_preds = []

# SCD(인덱스 1)의 기준을 0.05부터 0.5까지 바꿔가며 테스트
for th in np.arange(0.05, 0.55, 0.05):
    temp_preds = []
    for probs in avg_preds_proba:
        # 만약 SCD일 확률이 설정한 기준(th)보다 높으면 -> 무조건 SCD(1)로 판단
        if probs[1] >= th:
            temp_preds.append(1) # SCD Class Index = 1
        else:
            # 아니면 원래대로 확률 1등인 클래스 선택
            temp_preds.append(np.argmax(probs))

    # 점수 계산
    score = f1_score(y_test, temp_preds, average='macro')

    # 점수가 오르면 기록
    if score > best_score:
        best_score = score
        best_th = th
        final_optimized_preds = temp_preds

print(f"\n SCD 최적 임계값: {best_th:.2f}")
print(f"   (SCD 확률이 {best_th*100:.0f}%만 넘어도 SCD라고 판단합니다)")

# ==========================================
# 9. 최적화된 최종 점수
# ==========================================
print("\n" + "="*40)
print(f" 보정 후 최종 Macro F1 Score: {best_score:.4f}")
print("="*40)
print(classification_report(y_test, final_optimized_preds, target_names=['CN', 'SCD', 'MCI', 'Dem'], zero_division=0))


🔧 SCD(주관적 인지저하) 클래스 구출 작전을 시작합니다...

✨ 찾았다! SCD 최적 임계값: 0.30
   (SCD 확률이 30%만 넘어도 SCD라고 판단합니다)

🏆 보정 후 최종 Macro F1 Score: 0.6022
              precision    recall  f1-score   support

          CN       0.87      0.81      0.84        91
         SCD       0.17      0.40      0.24         5
         MCI       0.72      0.72      0.72        46
         Dem       0.67      0.57      0.62         7

    accuracy                           0.76       149
   macro avg       0.61      0.63      0.60       149
weighted avg       0.79      0.76      0.77       149



In [ ]:
import os

# 1. 저장할 폴더 만들기
save_path = '/content/drive/MyDrive/AiWemeet/models/'
if not os.path.exists(save_path):
    os.makedirs(save_path)
    print(f" 폴더 생성 완료: {save_path}")

print(" 모델 저장 시작")

# 2. 5번 반복하며 모델 재학습 및 저장
seeds = [42, 2023, 2024, 777, 999]

for i, seed in enumerate(seeds):
    filename = f'tabnet_model_seed_{seed}'

    # 모델 정의
    clf = TabNetClassifier(
        n_d=64, n_a=64, n_steps=5, gamma=1.2, n_independent=2, n_shared=2, lambda_sparse=1e-4,
        optimizer_fn=torch.optim.Adam, optimizer_params=dict(lr=1e-2),
        scheduler_params={"step_size":20, "gamma":0.9}, scheduler_fn=torch.optim.lr_scheduler.StepLR,
        mask_type='entmax', seed=seed, verbose=0
    )

    # 학습
    clf.fit(
        X_train=X_train_res, y_train=y_train_res,
        eval_set=[(X_valid, y_valid)], eval_name=['valid'], eval_metric=['balanced_accuracy'],
        max_epochs=150, patience=40, batch_size=256, virtual_batch_size=32,
        num_workers=0, drop_last=False, weights=1
    )

    # 저장
    saved_filepath = clf.save_model(save_path + filename)
    print(f"  [{i+1}/5] 저장 완료: {saved_filepath}")

print("\n 모든 모델 파일이 구글 드라이브에 저장됨")

📁 폴더 생성 완료: /content/drive/MyDrive/AiWemeet/models/
💾 모델 저장을 시작합니다...

Early stopping occurred at epoch 85 with best_epoch = 45 and best_valid_balanced_accuracy = 0.61728


/usr/local/lib/python3.12/dist-packages/pytorch_tabnet/callbacks.py:172: UserWarning: Best weights from best epoch are automatically used!
  warnings.warn(wrn_msg)


Successfully saved model at /content/drive/MyDrive/AiWemeet/models/tabnet_model_seed_42.zip
  [1/5] 저장 완료: /content/drive/MyDrive/AiWemeet/models/tabnet_model_seed_42.zip

Early stopping occurred at epoch 86 with best_epoch = 46 and best_valid_balanced_accuracy = 0.65321


/usr/local/lib/python3.12/dist-packages/pytorch_tabnet/callbacks.py:172: UserWarning: Best weights from best epoch are automatically used!
  warnings.warn(wrn_msg)


Successfully saved model at /content/drive/MyDrive/AiWemeet/models/tabnet_model_seed_2023.zip
  [2/5] 저장 완료: /content/drive/MyDrive/AiWemeet/models/tabnet_model_seed_2023.zip

Early stopping occurred at epoch 70 with best_epoch = 30 and best_valid_balanced_accuracy = 0.57894


...

Successfully saved model at /content/drive/MyDrive/AiWemeet/models/tabnet_model_seed_2024.zip
  [3/5] 저장 완료: /content/drive/MyDrive/AiWemeet/models/tabnet_model_seed_2024.zip

Early stopping occurred at epoch 91 with best_epoch = 51 and best_valid_balanced_accuracy = 0.61749


/usr/local/lib/python3.12/dist-packages/pytorch_tabnet/callbacks.py:172: UserWarning: Best weights from best epoch are automatically used!
  warnings.warn(wrn_msg)


Successfully saved model at /content/drive/MyDrive/AiWemeet/models/tabnet_model_seed_777.zip
  [4/5] 저장 완료: /content/drive/MyDrive/AiWemeet/models/tabnet_model_seed_777.zip

Early stopping occurred at epoch 89 with best_epoch = 49 and best_valid_balanced_accuracy = 0.6033


/usr/local/lib/python3.12/dist-packages/pytorch_tabnet/callbacks.py:172: UserWarning: Best weights from best epoch are automatically used!
  warnings.warn(wrn_msg)


Successfully saved model at /content/drive/MyDrive/AiWemeet/models/tabnet_model_seed_999.zip
  [5/5] 저장 완료: /content/drive/MyDrive/AiWemeet/models/tabnet_model_seed_999.zip

✨ 모든 모델 파일이 구글 드라이브에 저장되었습니다!
